In [ ]:
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 11.8 MB/s eta 0:00:00


In [ ]:
!pip install langchain

In [ ]:
!pip install -q langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.7/409.7 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.8 MB/s eta 0:00:00


In [ ]:
!pip install openai

In [ ]:
!pip install --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 6.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
import gradio as gr
import openai
import subprocess
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import mnist
import numpy as np
from langchain.llms import OpenAI
import openai
from openai import OpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

with open("/content/drive/MyDrive/muse_lab/open_key.txt", "r") as file:
    api_key = file.read().strip()

In [ ]:
# OpenAI API 키를 환경변수로 설정
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
# GPT 작동 테스트
client = OpenAI(
    api_key=api_key
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "hi, nice to meet you",
        }
    ],
    model="gpt-4o",
)
print(chat_completion.choices[0].message.content)

Hello! Nice to meet you too. How can I assist you today?


In [ ]:
import os
import openai
import gradio as gr
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# 모델 구성 저장
model_layers = []
preprocessing_steps = []

# CNN 모델에 적합한 레이어 옵션
layer_options = ["Conv2D", "MaxPooling2D", "Flatten", "Dense", "Dropout", "BatchNormalization"]


# 글로벌 변수
loaded_data = None

# 데이터셋 불러오기 함수
def load_dataset(dataset_name):
    global loaded_data
    if dataset_name == "MNIST":
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        loaded_data = (X_train, y_train, X_test, y_test)
        return f"{dataset_name} 데이터셋이 성공적으로 불러와졌습니다! ({X_train.shape[0]}개 학습 데이터)"
    else:
        return "지원되지 않는 데이터셋입니다."

# 전처리 함수
def preprocess_data(apply_normalization, add_channel, one_hot_encode):
    global loaded_data
    if loaded_data is None:
        return "먼저 데이터셋을 불러오세요!"

    X_train, y_train, X_test, y_test = loaded_data

    if apply_normalization:
        X_train = X_train.astype('float32') / 255.0
        X_test = X_test.astype('float32') / 255.0

    if add_channel:
        X_train = X_train[..., None]
        X_test = X_test[..., None]

    if one_hot_encode:
        y_train = to_categorical(y_train, num_classes=10)
        y_test = to_categorical(y_test, num_classes=10)

    loaded_data = (X_train, y_train, X_test, y_test)
    return "전처리가 완료되었습니다!"


# 레이어 추가 함수
def add_layer(layer_type, params):
    try:
        parsed_params = {}
        if params.strip():
            param_pairs = params.split(",")
            for pair in param_pairs:
                key, value = pair.split("=")
                parsed_params[key.strip()] = eval(value.strip())

        model_layers.append((layer_type, parsed_params))
        return generate_model_blocks(), "레이어가 성공적으로 추가되었습니다!"
    except Exception as e:
        return generate_model_blocks(), f"에러 발생: {str(e)}"

# 모델 초기화 함수
def reset_model():
    global model_layers
    model_layers = []
    return generate_model_blocks(), "모델이 초기화되었습니다."

# HTML 기반 모델 블록 생성 함수
def generate_model_blocks():
    html = '''
    <div style="display: flex; flex-direction: column; align-items: center; gap: 15px; padding: 20px;">
    '''
    for idx, (layer_type, params) in enumerate(model_layers):
        html += f"""
        <div style="
            border: 2px solid #2E2E2E;
            background-color: #333333;
            color: #EAEAEA;
            padding: 20px;
            margin: 10px;
            width: 320px;
            text-align: center;
            border-radius: 12px;
            box-shadow: 0 6px 12px rgba(0, 0, 0, 0.2), 0 3px 6px rgba(0, 0, 0, 0.15);
        ">
            <h3 style="margin: 0; color: #FFD700;">{idx + 1}. {layer_type}</h3>
            <p style="margin: 10px 0 0; font-size: 14px; line-height: 1.6;">{params}</p>
        </div>
        """
    html += '</div>'
    return html

# 코드 변환 함수
def convert_to_code(epochs=5):
    """
    AST를 기반으로 Keras 코드를 생성.
    """
    code = "from tensorflow.keras.models import Sequential\n"
    code += "from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization\n"
    code += "from tensorflow.keras.datasets import mnist\nimport numpy as np\n\n"

    # 데이터 전처리 추가
    code += "# 데이터 로드 및 전처리\n"
    code += "(X_train, y_train), (X_test, y_test) = mnist.load_data()\n"
    code += "X_train = X_train.reshape(-1, 28, 28, 1)  # 4D 텐서로 변환\n"
    code += "X_test = X_test.reshape(-1, 28, 28, 1)\n"
    code += "X_train, X_test = X_train / 255.0, X_test / 255.0  # 정규화\n"

    # 모델 정의 코드 추가
    code += "\nmodel = Sequential()\n"
    for layer_type, params in model_layers:
        params_str = ", ".join([f"{k}={v}" for k, v in params.items()])
        code += f"model.add({layer_type}({params_str}))\n"

    # 에포크 반영
    code += f"""
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs={epochs}, validation_data=(X_test, y_test))
    print("Training Complete. Final Accuracy:", history.history['accuracy'][-1])
    """
    return code.strip()  # Markdown 구문 제거



# GPT API로 설명 요청
def explain_model(model_code, train_results):
    prompt = "다음은 생성된 Keras 모델 코드와 학습 결과입니다:\n\n"
    prompt += "### 모델 코드 ###\n"
    prompt += model_code + "\n\n"
    prompt += "### 학습 결과 ###\n"
    prompt += train_results + "\n\n"
    prompt += "위 내용을 바탕으로 이 모델의 구조와 성능 분석, 그리고 모델 튜닝 방법을 초보자가 이해할 수 있도록 설명해 주세요."
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system",
                 "content": "You are an expert in Python and Keras." + prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"GPT 요청 중 에러 발생: {str(e)}"


# GPT API로 코드 요청 및 반환
def request_gpt_code(raw_code):
    prompt = f"다른 코멘트나 글은 절대 적지 않고 오직 요구하는 파이썬 코드만을 작성해주세요. 다음 Keras 코드를 데이터에 맞게 실행 가능한 형태로 수정해 주세요:\n\n{raw_code}. "
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system",
                 "content": "You are an expert in Python and Keras." + prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"GPT 요청 중 에러 발생: {str(e)}"


# 모델 훈련 함수
def train_model(gpt_code):
    """
    GPT가 반환한 Keras 코드를 실행하여 학습.
    """
    try:
        file_path = "/content/model_training.py"
        with open(file_path, "w") as f:
            f.write(gpt_code)

        result = subprocess.check_output(["python3", file_path], stderr=subprocess.STDOUT, text=True)
        return result
    except subprocess.CalledProcessError as e:
        return f"훈련 중 에러 발생:\n{e.output}"


def get_last_two_sentences(output_text):
    """
    주어진 텍스트에서 마지막 두 문장을 반환.
    """
    sentences = output_text.strip().split(".")
    # 마지막 두 문장 추출 및 공백 제거
    last_two = ". ".join(sentences[-3:]).strip()
    return last_two if last_two else "훈련 결과가 비어 있습니다."


# Gradio UI 정의
with gr.Blocks() as demo:
    gr.Markdown("# CNN 노코드 AI 블록 모델 생성기")

    # 데이터셋 불러오기
    dataset_dropdown = gr.Dropdown(["MNIST"], label="데이터셋 선택")
    load_button = gr.Button("데이터셋 불러오기")
    dataset_status = gr.Textbox(label="상태", interactive=False)

    load_button.click(load_dataset, inputs=dataset_dropdown, outputs=dataset_status)

    # 전처리 옵션
    gr.Markdown("### 전처리 옵션")
    apply_normalization = gr.Checkbox(label="Normalize (0~1로 스케일)", value=True)
    add_channel = gr.Checkbox(label="Add Channel Dimension (28x28 → 28x28x1)", value=True)
    one_hot_encode = gr.Checkbox(label="One-Hot Encode Labels", value=True)

    preprocess_button = gr.Button("전처리 실행")
    preprocess_status = gr.Textbox(label="전처리 상태", interactive=False)

    preprocess_button.click(preprocess_data, inputs=[apply_normalization, add_channel, one_hot_encode], outputs=preprocess_status)

    # 모델 구성
    layer_type = gr.Dropdown(choices=layer_options, label="레이어 타입")
    params_input = gr.Textbox(label="매개변수 (예: filters=32, kernel_size=(3,3), activation='relu')")
    add_layer_button = gr.Button("레이어 추가")
    reset_model_button = gr.Button("모델 초기화")
    model_blocks = gr.HTML()

    # 에포크 입력 필드
    epochs_input = gr.Number(label="훈련 에포크 수", value=5)  # 기본값 5


    # 코드 생성 및 GPT 요청, 훈련
    generate_code_button = gr.Button("코드 생성")
    code_output = gr.Code(label="Keras 코드")
    gpt_request_button = gr.Button("GPT 사용하여 실행 가능한 코드로 변환")
    gpt_code_output = gr.Code(label="GPT 변환 코드")
    train_model_button = gr.Button("모델 학습")
    train_output = gr.Textbox(label="훈련 결과")


    # GPT와의 연결 및 코드 생성
    explain_button = gr.Button("모델 설명 및 성능 분석")
    explanation_output = gr.Textbox(label="GPT 설명")

    # 레이어 추가 버튼 동작
    add_layer_button.click(
        fn=add_layer,
        inputs=[layer_type, params_input],
        outputs=[model_blocks]
    )

    reset_model_button.click(
        fn=reset_model,
        outputs=[model_blocks]
    )

    # 코드 생성 버튼 동작 수정
    generate_code_button.click(
    fn=lambda epochs: convert_to_code(epochs),  # 에포크 전달
    inputs=epochs_input,  # 에포크 입력 필드 추가
    outputs=code_output
)

    # GPT 코드 요청 버튼 동작
    gpt_request_button.click(
        fn=request_gpt_code,
        inputs=code_output,
        outputs=gpt_code_output
    )

    # 모델 학습 버튼 동작
    train_model_button.click(
        fn=train_model,
        inputs=gpt_code_output,
        outputs=train_output
    )

    # GPT 설명 요청
    explain_button.click(
        fn=lambda code, train: explain_model(code, get_last_two_sentences(train)),
        inputs=[gpt_code_output, train_output],  # gpt_code_output과 축약된 train_output 전달
        outputs=explanation_output
    )

    # 코드 생성 버튼 동작 수정
    generate_code_button.click(
    fn=lambda epochs: convert_to_code(epochs),  # 에포크 전달
    inputs=epochs_input,  # 에포크 입력 필드 추가
    outputs=code_output
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9cd5cd905ceb4e2967.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
